Group Name: AG 16.

| Name | Student ID |
|----------|----------|
| LIU Yichao | A0304386A |
| LI Yizhe | A0304167J |
| LEE Bing Cheng | A0297134Y |
| #YOUR ANSWER GOES HERE | #YOUR ANSWER GOES HERE |
| #YOUR ANSWER GOES HERE | #YOUR ANSWER GOES HERE |

# Question 1

Consider the maze shown below. The Maze has 16 rows and 24 columns The objective is to find a shortest path from cell $S$ to cell $G$.


![Maze](Maze.jpg)


The agent can take four actions in each cell: 'RIGHT', 'DOWN', 'UP', 'LEFT'.  

Each cell is represented as $(x,y)$, where $x$ indicates row number and $y$ indicates column number. Action 'UP' takes the agent from cell $(x,y)$ to $(x+1,y)$. Action 'DOWN' takes the agent from cell $(x,y)$ to $(x-1,y)$. Action 'RIGHT' takes the agent from cell $(x,y)$ to $(x,y+1)$. Action 'LEFT' takes the agent from cell $(x,y)$ to $(x,y-1)$. The triplet $(s,a,s')$  indicates that taking action $a$ at state $s$ leads to state $s'$. Actions 'LEFT' or 'RIGHT' cost 10 units for all $(s,a,s')$. Actions 'UP' or 'DOWN' cost 1 unit for all  $(s,a,s')$.  The agent cannot move into cells that are shaded. Assume that the agent knows the boundaries of the maze and has full observability. Consequently, at the bottom (row 0) and top (row 15), the agent will not take actions 'DOWN' and 'UP', respectively; at left (column 0) and right (column 23) columns, the agent will not take 'LEFT' and 'RIGHT' actions, respectively. Similalry, the agent will not take actions that lead to shaded region in the maze.

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the form of a table.
 
3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.
 

Note 1: You must follow the problem formulation discussed in the class. A abstract class for Problem amd Node definition is presented below. The search tree generation should follow the template discussed in the class (i.e., Node class, expand methods, etc.). 

Note 2: The code should be written in a single jupyter notebook file.

In [1]:
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [2]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 


In [3]:
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
  
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [4]:
from collections import deque
import heapq

FIFOQueue = deque
LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

## **Q1.a: Class Maze(Problem)** [5 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms.

In [5]:
MAZE_ROWS = 16
MAZE_COLUMNS = 24
ACTIONS = {'UP', 'LEFT', 'RIGHT', 'DOWN'}
SHADED = {(14,9),(10,9),(9,9),(8,9),(7,9),(6,9),(13,10),(12,10),(11,10),(10,10),(11,12),(10,12),(11,13),(10,13)}

class Maze(Problem):
    
    def __init__(self, initial=None, goal=None):
        Problem.__init__(self, initial=initial, goal=goal)
        #self.actions=('UP','DOWN','LEFT','RIGHT')                   
        #directions = ((1,0),(-1,0),(0,-1),(0,1))

    def h1(self, node): #Euclidean distance
        #Your Code goes here
        gx,gy = self.goal 
        x,y = node.state
        heu =  math.sqrt((gx-x)**2+(gy-y)**2)
        return heu


    def h2(self, node): #manhattan
        #Your Code goes here
        gx,gy = self.goal 
        x,y = node.state
        heu = abs(gx-x)+abs(gy-y)
        return heu

    def action_cost(self, s, a, s1): 
        #Your Code goes here
        if self.result(s,a) == s1:
            if a in {'UP','DOWN'}:
                return 1
            else:
                return 10
        return False
        
                  
    def result(self, state, action):        
        #Your Code goes here  
        if action not in ACTIONS:
            return False
        if action == 'UP':
            dx,dy = 1,0
        elif action == 'DOWN':
            dx,dy = -1,0
        elif action == 'LEFT':
            dx,dy = 0,-1
        elif action == 'RIGHT':
            dx,dy = 0,1
        x,y = state
        result_x,result_y = dx+x,dy+y
        if 0 <= result_x < MAZE_ROWS and 0 <= result_y < MAZE_COLUMNS and (result_x,result_y) not in SHADED:
            return (result_x,result_y)
        return False


    def actions(self, state):       
        #Your Code goes here   
        available_actions = set()
        for action in ACTIONS:
            if self.result(state,action):
                available_actions.add(action)
        return available_actions

    


### Testing the model

In [6]:
#Testing Maze class
#Note that we perform additional tests to validate the modeling

#An instance of Maze problem; initialize the Maze class with appropriate values
m1 = Maze(initial = None, goal = None)

# Testing the methods in Maze class
assert m1.result((4,12),'UP') == (5,12)
assert m1.action_cost((4,12),'UP',(5,12)) == 1
assert m1.actions((4,12)) == {'UP', 'LEFT', 'RIGHT', 'DOWN'} #Returning a set means that results may vary across devices / python versions as we looped over the set in Node.

## **Q1.b: Analysis of the Algorithms** [5 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

In [7]:
#Your code for breadth-first search


maze = Maze(initial=(8,10),goal=(11,9))


def breadth_first_search(maze):
    max_frontier = 0
    expanded_nodes = 0
    node = Node(maze.initial)
    if maze.goal == node.state:
        return node
    frontier = FIFOQueue([node])
    reached = {maze.initial}
    while len(frontier) != 0:
        node = frontier.popleft()
        expanded_nodes += 1
        for child in expand(maze, node):
            s = child.state
            if maze.goal == s:
                return node,max_frontier,expanded_nodes
            if s not in reached:
                reached.add(s)
                frontier.append(child)
                if len(frontier) > max_frontier:
                    max_frontier = len(frontier)
    return False

node,mf,en= breadth_first_search(maze)
print(f"Path cost: ",node.path_cost)
print(f"Max frontier: ",mf)
print(f"Expanded nodes: ",en)

Path cost:  29
Max frontier:  30
Expanded nodes:  159


In [11]:
#Your code for  depth-first search with cycle-check

maze = Maze(initial=(10,8),goal=(11,9))

# def is_cycle(node):
#     ancestor = node.parent
#     while ancestor != None:
#         if ancestor.state == node.state: #Parent
#             #print(ancestor.state,node.state)
#             return True
#         ancestor = ancestor.parent
#     return False

# def depth_first_search(maze):
#     max_frontier = 0
#     expanded_nodes = 0
#     node = Node(maze.initial)
#     if maze.goal == node.state:
#         return node
#     frontier = LIFOQueue([node])
#     while len(frontier) != 0:
#         node = frontier.pop()
#         print(node.state)
#         if maze.goal == node.state:
#             return node,max_frontier,expanded_nodes
#         if not is_cycle(node):
#             expanded_nodes+=1
#             for child in expand(maze,node):
#                 frontier.append(child)
#                 if len(frontier) > max_frontier:
#                     max_frontier = len(frontier)
#     return False

# def depth_first_search(maze):
#     max_frontier = 0
#     expanded_nodes = 0
#     node = Node(maze.initial)
#     if maze.goal == node.state:
#         return node
#     frontier = LIFOQueue([node])
#     visited ={node.state}
#     while len(frontier) != 0:
#         node = frontier.pop()
#         if maze.goal == node.state:
#             return node,max_frontier,expanded_nodes
#         expanded_nodes+=1
#         for child in expand(maze,node):
#             if child.state not in visited:
#                 frontier.append(child)
#                 visited.add(child.state)
#                 if len(frontier) > max_frontier:
#                     max_frontier = len(frontier)
#     return False

node,mf,en = depth_first_search(maze)
print(f"Path cost: ",node.path_cost)
print(f"Max frontier: ",mf)
print(f"Expanded nodes: ",en)

NameError: name 'depth_first_search' is not defined

In [12]:
#Your code for  iterative Deepening search with cycle-check

maze = Maze(initial=(8,10),goal=(11,9))

def iterative_depth_first_search(maze,depth):
    max_frontier = 0
    expanded_nodes = 0
    result = failure
    node = Node(maze.initial)
    frontier = LIFOQueue([node])
    while len(frontier) != 0:
        node = frontier.pop()
        if maze.goal == node.state:
            return node,max_frontier,expanded_nodes
        if len(node) > depth:
            continue
        else:
            if not is_cycle(node):
                expanded_nodes+=1
                for child in expand(maze,node):
                    frontier.append(child)
                    if len(frontier) > max_frontier:
                        max_frontier = len(frontier)
    return result,max_frontier,expanded_nodes

for i in range(1,100):
    node,mf,en = iterative_depth_first_search(maze,i)
    print("____________________")
    print("Depth: ",i)
    print(f"Path cost: ",node.path_cost)
    print(f"Max frontier: ",mf)
    print(f"Expanded nodes: ",en)
    print("____________________")
    if node.path_cost != math.inf:
        break

NameError: name 'is_cycle' is not defined

In [13]:
#Your code for uniform-cost search

maze = Maze(initial=(8,10),goal=(11,9))

def get_path_cost(node):
    return node.path_cost

def uniform_cost_search(maze,failure):
    max_frontier = 0
    expanded_nodes = 0
    node = Node(maze.initial)
    frontier = PriorityQueue([node],get_path_cost)
    reached = {}
    reached[node.state] = node
    while len(frontier) != 0:
        node = frontier.pop()
        if node.state == maze.goal:
            temp = node
            temp_l = deque()
            while temp != None:
                temp_l.append(temp.state)
                temp = temp.parent
            temp_l.reverse()
            return node,max_frontier,expanded_nodes
        expanded_nodes += 1
        for child in expand(maze, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
                if len(frontier) > max_frontier:
                    max_frontier = len(frontier)
    return failure,0,0

node,mf,en = uniform_cost_search(maze,failure)
print(f"Path cost: ",node.path_cost)
print(f"Max frontier: ",mf)
print(f"Expanded nodes: ",en)

Path cost:  39
Max frontier:  34
Expanded nodes:  91


In [14]:
#Your code for A* Search

def a_star_h1(node):
    return maze.h1(node) + node.path_cost

maze = Maze(initial=(8,10),goal=(11,9))

def a_star_search_h1(maze,failure):
    max_frontier = 0
    expanded_nodes = 0
    node = Node(maze.initial)
    frontier = PriorityQueue([node],a_star_h1)
    reached = {}
    reached[node.state] = node
    while len(frontier) != 0:
        node = frontier.pop()
        if node.state == maze.goal:
            return node,max_frontier,expanded_nodes
        expanded_nodes += 1
        for child in expand(maze, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
                if len(frontier) > max_frontier:
                    max_frontier = len(frontier)
    return failure,0,0

node,mf,en = a_star_search_h1(maze,failure)
print(f"h1")
print(f"Path cost: ",node.path_cost)
print(f"Max frontier: ",mf)
print(f"Expanded nodes: ",en)


def a_star_h2(node):
    return maze.h2(node) + node.path_cost

maze = Maze(initial=(8,10),goal=(11,9))

def a_star_search_h2(maze,failure):
    max_frontier = 0
    expanded_nodes = 0
    node = Node(maze.initial)
    frontier = PriorityQueue([node],a_star_h2)
    reached = {}
    reached[node.state] = node
    while len(frontier) != 0:
        node = frontier.pop()
        if node.state == maze.goal:
            return node,max_frontier,expanded_nodes
        expanded_nodes += 1
        for child in expand(maze, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
                if len(frontier) > max_frontier:
                    max_frontier = len(frontier)
    return failure,0,0

node,mf,en = a_star_search_h2(maze,failure)
print(f"h2")
print(f"Path cost: ",node.path_cost)
print(f"Max frontier: ",mf)
print(f"Expanded nodes: ",en)
        

h1
Path cost:  39
Max frontier:  34
Expanded nodes:  67
h2
Path cost:  39
Max frontier:  34
Expanded nodes:  65


In [15]:
#Your code for greedy-best first search

maze = Maze(initial=(8,10),goal=(11,9))

def best_first_search(maze,failure):
    max_frontier = 0
    expanded_nodes = 0
    node = Node(maze.initial)
    frontier = PriorityQueue([node],maze.h1)
    reached = {}
    reached[node.state] = node
    while len(frontier) != 0:
        node = frontier.pop()
        if node.state == maze.goal:
            return node,max_frontier,expanded_nodes
        expanded_nodes += 1
        for child in expand(maze, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
                if len(frontier) > max_frontier:
                    max_frontier = len(frontier)
    return failure,0,0

node,mf,en = best_first_search(maze,failure)
print(f"h1")
print(f"Path cost: ",node.path_cost)
print(f"Max frontier: ",mf)
print(f"Expanded nodes: ",en)
        

def best_first_search(maze,failure):
    max_frontier = 0
    expanded_nodes = 0
    node = Node(maze.initial)
    frontier = PriorityQueue([node],maze.h2)
    reached = {}
    reached[node.state] = node
    while len(frontier) != 0:
        node = frontier.pop()
        if node.state == maze.goal:
            return node,max_frontier,expanded_nodes
        expanded_nodes += 1
        for child in expand(maze, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
                if len(frontier) > max_frontier:
                    max_frontier = len(frontier)
    return failure,0,0

node,mf,en = best_first_search(maze,failure)
print(f"h2")
print(f"Path cost: ",node.path_cost)
print(f"Max frontier: ",mf)
print(f"Expanded nodes: ",en)
        

h1
Path cost:  61
Max frontier:  17
Expanded nodes:  22
h2
Path cost:  61
Max frontier:  17
Expanded nodes:  22


2. Identify the number of number of expanded nodes, maximum frontier size, and path-cost for the above algorithms.  Summarize the statistics in the following table.


 Algorithm             |Number of expanded nodes  | Maximum Frontier Size  |  Path Cost                  |
|:---------------------|:-------------------------|:-----------------------|:----------
| Breadth-First Search |            175              |       31                 | 39
| DFS with cycle check |          333                |         590               | 2913
| IDS with cycle check |           688               |         33               | 39
| UCS                  |          90               |          34              | 39
| A* Search            |          67                |            34            | 39
| GBFS                 |          22                |          17              | 61

3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.


 Algorithm             |Number of expanded nodes  | Maximum Frontier Size  |  Path Cost                  |
|:---------------------|:-------------------------|:-----------------------|:----------
| Greedy best first search: Euclidean distance |            22             |       17                 | 61
| Greedy best first search: Manhattan distance |             22               |      17                    | 61
| A* search: Euclidean distance |            67             |       34                 | 39
| A* search: Manhattan distance |             67               |      34                    | 39



In this scenario, A* Search performed better.

This is because Greedy best first Search uses only heuristic information and focuses on getting to the goal state, resulting in a smaller frontier size and fewer expanded nodes.
However, the hereuistic information was not able to provide an optimal path as the optimal path required the heuristic to increase in value initially.

On the other hand, A*Search considers both the cost up to the state of the node, as well as the heuristic information. 
Similar to Greedy best first search, it will initially expand in the non-optimal path due to the influence of the heuristic information.
However, it would choose to expand other paths when the path cost plus heuristic cost is no longer the lowest when going along the non-optimal path.
This results in the expanded nodes and maximum frontier size being higher than greedy best first search since it will keep switching paths but A*Search is able to identify the optimal path.